In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import folium
from folium.plugins import HeatMap
import json
import requests
from shapely.geometry import Polygon
from shapely.ops import transform
from pyproj import Proj, transform as proj_transform
import plotly.graph_objs as go
import plotly.offline as pyo

In [ ]:
def capitals():
    '''
    Generates HTML file that displays interactive map with markers for each capital city
    
    Args: None
    
    Returns: None
    '''
    df = pd.read_csv('capitals_lat_lon.csv')
    
#     #check dataset
#     print(df.head())
#     print(df.info())
    
    #create folium map centered around mean lat and lon
    map_centre = [df['Latitude'].mean(), df['Longitude'].mean()]
    map_1 = folium.Map(location = map_centre, zoom_start = 2)
    
    #add marker for each capital city
    for _, row in df.iterrows():
        folium.Marker(
            location=[row['Latitude'], row['Longitude']],
            popup=row['Capital']
        ).add_to(map_1)
        
    #html file
    map_1.save('capitals_map_nesalin.html')
    
if __name__ == "__main__":
    capitals()

In [ ]:
def triangulation(p1 = None, p2 = None, p3 = None):
    '''
    Displays a triangle on a map and computes it's perimeter and area
    
    Args:
        p1: Longitude and latitude of the first point (default is None)
        p2: Longitude and latitude of the first point (default is None)
        p3: Longitude and latitude of the first point (default is None)
        
    Returns: Area and perimeter of the triangle
    '''
    
    #default points
    default_points = [
        (-6.8498, 33.9716),  #Rabat
        (18.4241, -33.9249), #Cape Town
        (45.3182, 2.0469)    #Mogadishu
    ]
    
    #ask for user input
    if p1 is None or p2 is None or p3 is None:
        print("Input latitude and longitude for three points forming a triangle covering Africa:")
        for i in range(3):
            lat = float(input(f"Enter latitude for point {i + 1}: "))
            lon = float(input(f"Enter longitude for point {i + 1}: "))
            if i == 0:
                p1 = (lon, lat)
            elif i == 1:
                p2 = (lon, lat)
            elif i == 2:
                p3 = (lon, lat)
    else:
        p1 = p1 if p1 else default_points[0]
        p2 = p2 if p2 else default_points[1]
        p3 = p3 if p3 else default_points[2]
        
    #create folium map
    m = folium.Map(location=[0, 20], zoom_start=3)
    
    for point in [p1, p2, p3]:
        folium.Marker(location=(point[1], point[0])).add_to(m)
    
    #form a triangle
    folium.PolyLine(locations=[(p1[1], p1[0]), (p2[1], p2[0]), (p3[1], p3[0]), (p1[1], p1[0])], color='blue').add_to(m)
    
    #html
    m.save('africa_triangle_nesalin.html')
    
    #area and perimeter calculation
    polygon = Polygon([p1, p2, p3])
    
    area = polygon.area
    perimeter = polygon.length
    
    print(f"Triangle area: {area} square degrees")
    print(f"Triangle perimeter: {perimeter} degrees")
    
    return area, perimeter

if __name__ == "__main__":
    triangulation()

In [ ]:
def states(kansas, nebraska):
    '''
    Constructs GeoJSON file from boundaries given for Kansas and Nebraska
    
    Args:
        kansas: list with 4 tuples representing Kansas boundary
        nebraska: list with 4 tuples representing Nebraska boundary
        
    Returns: None
    '''
    geojson_data = {
        "type": "FeatureCollection",
        "features": [
            {
                "type": "Feature",
                "geometry": {
                    "type": "Polygon",
                    "coordinates": [kansas]
                },
                "properties": {
                    "name": "Kansas"
                }
            },
            {
                "type": "Feature",
                "geometry": {
                    "type": "Polygon",
                    "coordinates": [nebraska]
                },
                "properties": {
                    "name": "Nebraska"
                }
            }
        ]
    }
    
    with open('kansas_nebraska_nesalin.geojson', 'w') as f:
        json.dump(geojson_data, f)
    
def plot_states(file):
    '''
    Reads GeoJSON file and plots result
    
    Args:
        file (str): name of GeoJSON file to read and plot
        
    Returns: None
    '''
    
    with open (file, 'r') as f:
        data = json.load(f)
        
#     #check data
#     print(json.dumps(data, indent=2))
        
    m = folium.Map(location=[39.5, -98.5], zoom_start=5)
    
    folium.GeoJson(data).add_to(m)
    
    m.save('states_map_nesalin.html')
    
kansas = [
[-102.051744, 40.003166],
[-94.588421, 40.003166],
[-94.588421, 36.993076],
[-102.051744, 36.993076]
]

nebraska = [
    [-104.053514, 41.003444],
    [-95.30829, 41.003444],
    [-95.30829, 40.003166],
    [-95.30829, 39.003444],
    [-104.053514, 39.003444],
    [-104.053514, 40.003444]
]

if __name__ == "__main__":
    states(kansas, nebraska)

    plot_states('kansas_nebraska_nesalin.geojson')
    

In [ ]:
def if_youre_happy_and_you_know_it():
    '''
    Generates html file that displays choropleth map of US showing happiness index by states
    
    Args: None
    
    Returns: None
    '''
    with open('us-states.json') as f:
        geojson_data = json.load(f)
        
#         #check dataset
#     print(geojson_data.keys())
#     print(geojson_data['features'][0])

    #extract state names
    states = [feature['properties']['name'] for feature in geojson_data['features']]
    
#     #print list of state names
#     print(states)
    
    #generate random happiness index for states
    happiness_index = np.random.rand(len(states)) * 100
    data = pd.DataFrame({
        'state': states,
        'happiness_index': happiness_index
    })
    
#     #print new df summary
#     print(data.head())
#     print(data.describe())

    #choropleth map
    map_2 = folium.Map(location=[37.8, -96], zoom_start=4)
    
    folium.Choropleth(
        geo_data = geojson_data,
        name='choropleth of the US',
        data=data,
        columns = ['state', 'happiness_index'],
        key_on='feature.properties.name',
        legend_name='Happiness Index'
    ).add_to(map_2)
        
    folium.LayerControl().add_to(map_2)
    
    #html file
    map_2.save('us_happiness_index_nesalin.html')

if __name__ == "__main__":
    if_youre_happy_and_you_know_it()

In [ ]:
def random_hist(n):
    '''
    Generates n points from a random exponential distribution and plots a histogram
    
    Args:
        n (int): number of points to generate
        
    Returns: None
    '''
    
    #parameters for exponential distribution
    scale = 1.0 #mean
    
    data = np.random.exponential(scale, n)
    
    #number of bins
    num_bins = 30
    
    #histogram
    hist_data = go.Histogram(
        x = data,
        nbinsx = num_bins
    )
    
    #layout
    layout = go.Layout(
        title = "Histogram of Exponential Distribution",
        xaxis = dict(title = "Valje"),
        yaxis = dict(title = "Count"))
    
    fig = go.Figure(data = [hist_data], layout = layout)
    
    #html
    pyo.plot(fig, filename = "random_exponential_hist_nesalin.html")
    
if __name__ == "__main__":
    random_hist(500)